<a href="https://colab.research.google.com/github/Ellisetr/Intelligent-Systems-and-Technologies/blob/main/%D0%98%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D1%83%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B_%D0%B8_%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8_1_2_%D0%9B%D0%B0%D0%B1%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка либы в облако:

In [ ]:
%pip install --index-url https://test.pypi.org/simple/ PyARMViz
%pip install apriori_python
%pip install efficient-apriori
%pip install fpgrowth_py

Проверка работы либы

In [ ]:
from PyARMViz import datasets
from PyARMViz import *

rules = datasets.load_shopping_rules()
PyARMViz.generate_rule_graph_plotly(rules)

Открытие и получение dataset:

In [ ]:
import pandas as pd

class Big_dataset:
    def __init__(self) -> None:
        self.df = pd.read_csv("dataset.csv", usecols=['InvoiceNo', 'Description'])#, index_col=0)
        self.df['Description'] = self.df['Description'].astype(str)
        print(self.df)
        print(self.df.shape[0])
        print(list(self.df.iloc[0])) 
        ...

    def agregate_transactions(self):

        self.agregated_transactions = {}
        for i in range(self.df.shape[0]):
            if list(self.df.iloc[i])[0] not in self.agregated_transactions.keys():
                self.agregated_transactions[list(self.df.iloc[i])[0]] = [list(self.df.iloc[i])[1]]
            else:
                self.agregated_transactions[list(self.df.iloc[i])[0]].append(list(self.df.iloc[i])[1])
        
        print(self.agregated_transactions)
        self.agregated_transactions = list(self.agregated_transactions.values())

Apriori для данных из репозитория:

In [ ]:
from apriori_python import apriori

class Apriori:
    def __init__(self, dataset) -> None:
        self.transactions = dataset.agregated_transactions

    def create_rules(self, min_sup: float, min_conf: float):
        self.freq_item_set, self.rules = apriori(self.transactions, minSup=min_sup, minConf=min_conf)

Efficeint Apriori:

In [ ]:
from efficient_apriori import apriori as ef_apriori

class Efficient_apriori(Apriori):

    def __init__(self, dataset) -> None:
        super().__init__(dataset)

    def create_rules(self, min_sup: float, min_conf: float):
        self.freq_item_set, self.rules = ef_apriori(self.transactions, min_support=min_sup, min_confidence=min_conf)

    def print_rules(self, lhs_len: int, rhs_len: int):
        rules_rhs = filter(lambda rule: len(rule.lhs) == lhs_len and len(rule.rhs) == rhs_len, self.rules)
        for rule in sorted(rules_rhs, key=lambda rule: rule.confidence): #Не сортирует
            print(rule)

FPGrowth:

In [ ]:
from fpgrowth_py import fpgrowth

class Fpgrowth(Apriori):
    def __init__(self, dataset) -> None:
        super().__init__(dataset)
    def create_rules(self, min_sup: float, min_conf: float):
        self.freq_item_set, self.rules = fpgrowth(self.transactions, minSupRatio=min_sup, minConf=min_conf)

Вызов алгоритмов:

In [ ]:
def standard_apriori(dataset):
    Apr = Apriori(dataset)
    Apr.create_rules(0.02, 0.6)
    print(Apr.rules)
    print(Apr.freq_item_set)
    return Apr.rules


def eff_apriori(dataset):
    Ef_apr = Efficient_apriori(dataset)
    Ef_apr.create_rules(0.02, 0.6)
    Ef_apr.print_rules(2, 1)
    return Ef_apr.rules
    

def fpg(dataset):
    Fp = Fpgrowth(dataset)
    Fp.create_rules(0.02, 0.6)
    #print(Fp.freq_item_set)
    print('RULES')
    rules = [] 
    for rule in Fp.rules:
        if rule not in rules:
            rules.append(rule)
    for r in rules:
        print(r)
    return Fp.rules

Тест алгоритмов:

In [ ]:
def main():
    Ds = Big_dataset()
    Ds.agregate_transactions()

    #fpg(Ds)
    eff_apriori(Ds)
    #standard_apriori(Ds)
    PyARMViz.generate_rule_graph_plotly(eff_apriori(Ds))
main()

       InvoiceNo                          Description
0         536365   WHITE HANGING HEART T-LIGHT HOLDER
1         536365                  WHITE METAL LANTERN
2         536365       CREAM CUPID HEARTS COAT HANGER
3         536365  KNITTED UNION FLAG HOT WATER BOTTLE
4         536365       RED WOOLLY HOTTIE WHITE HEART.
...          ...                                  ...
541904    581587          PACK OF 20 SPACEBOY NAPKINS
541905    581587         CHILDREN'S APRON DOLLY GIRL 
541906    581587        CHILDRENS CUTLERY DOLLY GIRL 
541907    581587      CHILDRENS CUTLERY CIRCUS PARADE
541908    581587        BAKING SET 9 PIECE RETROSPOT 

[541909 rows x 2 columns]
541909
['536365', 'WHITE HANGING HEART T-LIGHT HOLDER']


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{GREEN REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER } -> {PINK REGENCY TEACUP AND SAUCER} (conf: 0.700, supp: 0.021, lift: 22.642, conv: 3.233)
{GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY TEACUP AND SAUCER} -> {ROSES REGENCY TEACUP AND SAUCER } (conf: 0.852, supp: 0.021, lift: 19.714, conv: 6.486)
{PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER } -> {GREEN REGENCY TEACUP AND SAUCER} (conf: 0.894, supp: 0.021, lift: 21.909, conv: 9.061)
{GREEN REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER } -> {PINK REGENCY TEACUP AND SAUCER} (conf: 0.700, supp: 0.021, lift: 22.642, conv: 3.233)
{GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY TEACUP AND SAUCER} -> {ROSES REGENCY TEACUP AND SAUCER } (conf: 0.852, supp: 0.021, lift: 19.714, conv: 6.486)
{PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY TEACUP AND SAUCER } -> {GREEN REGENCY TEACUP AND SAUCER} (conf: 0.894, supp: 0.021, lift: 21.909, conv: 9.061)
